<a href="https://colab.research.google.com/github/sdgroeve/Machine_Learning_course_UGent_D012554_2025/blob/main/notebooks/YOLO_cancer_cell_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Cancer Cell Detection: YOLO

This code fine-tunes a YOLOv10 model for the detection of cancer cells in images.

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv10

In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

We will also install two additional packages: [`roboflow`](https://github.com/roboflow/roboflow-python) to download the dataset from [Roboflow Universe](https://universe.roboflow.com/), which we will use to train our model, and [`supervision`](https://github.com/roboflow/supervision), which we will use for visualizing the results.

In [ ]:
!pip install -q supervision roboflow

## Download pre-trained weights

YOLOv10 provides weight files pre-trained on the [COCO dataset](https://cocodataset.org/) in various sizes. Let's download them.

In [ ]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!ls -lh {HOME}/weights

## Download dataset from Roboflow Universe

Roboflow is a platform designed to simplify the process of creating, managing, and preprocessing computer vision datasets.

To fine-tune the YOLO model for cancer cell detection we use [this dataset](https://universe.roboflow.com/national-yang-ming-chiao-tung-university-jvm0d/cell-type-ss5wm).

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install -q roboflow

from google.colab import userdata
from roboflow import Roboflow

import os
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())
#ROBOFLOW_API_KEY = os.environ["ROBOFLOW_API_KEY"]
ROBOFLOW_API_KEY = "wZKgeiCJuOw9rVjBBi3h"

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("national-yang-ming-chiao-tung-university-jvm0d").project("cancer-cell-box")
version = project.version(2)
dataset = version.download("yolov8")

## Fine-tuning YOLO

The follwoing command fine-tunes YOLOv10 on a custom dataset using the Ultralytics YOLO CLI.

Let's break it down:

```bash
yolo task=detect mode=train epochs=10 batch=32 plots=True \
model={HOME}/weights/yolov10n.pt \
data={dataset.location}/data.yaml
```

- **`yolo task=detect`** → Specifies the task as **object detection**.
- **`mode=train`** → Runs YOLO in **training mode**.
- **`epochs=10`** → Trains for **10 epochs** (iterations over the dataset).
- **`batch=32`** → Uses a **batch size of 32**, meaning 32 images are processed per training step.
- **`plots=True`** → Enables **visualization** of training metrics (loss, accuracy, etc.).
- **`model={HOME}/weights/yolov10n.pt`** → Uses a **pretrained YOLOv10n model** stored in the `{HOME}/weights/` directory.  
   - **YOLOv10n** is the **nano** version, optimized for speed and efficiency.
- **`data={dataset.location}/data.yaml`** → Specifies the **dataset configuration file**, which includes:
   - Paths to training and validation images.
   - Class labels for the dataset.

In [ ]:
%cd {HOME}

!yolo task=detect mode=train epochs=10 batch=32 plots=True \
model={HOME}/weights/yolov10n.pt \
data={dataset.location}/data.yaml

YOLO creates several results files, including the fine-tuned model:

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
from IPython.display import Image, display
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

## Inference with Custom Model

Let's start by loading our newly trained model:

In [ ]:
from ultralytics import YOLOv10
import supervision as sv

model = YOLOv10(f'{HOME}/runs/detect/train/weights/best.pt')

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{dataset.location}/valid/images",
    annotations_directory_path=f"{dataset.location}/valid/labels",
    data_yaml_path=f"{dataset.location}/data.yaml"
)

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

Let's randomly select an image from our validation set and visualize the results.

In [ ]:
import random

random_image = random.choice(list(dataset.images.keys()))
random_image = dataset.images[random_image]

results = model(source=random_image, conf=0.25)[0]
detections = sv.Detections.from_ultralytics(results)

annotated_image = bounding_box_annotator.annotate(
    scene=random_image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)